# Baseline FLAIR Training - Part 1


## 1. Setup & Installation


In [ ]:
!pip install nnunetv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [ ]:
import shutil, os

# Paths used by nnUNet
paths_to_clear = [
    "/kaggle/working/nnUNet_raw_data_base",
    "/kaggle/working/nnUNet_raw_data",
    "/kaggle/working/nnUNet_preprocessed",
    "/kaggle/working/nnUNet_results"
]

for p in paths_to_clear:
    if os.path.exists(p):
        print(f"🗑️ Removing {p} ...")
        shutil.rmtree(p)
print("✅ All previous nnUNet folders cleared.")

✅ All previous nnUNet folders cleared.


In [ ]:
!ls /kaggle/input

nnUNet_preprocessed  nnUNet_raw_data_base


In [ ]:
import os, shutil, json, nibabel as nib, subprocess
from pathlib import Path

# ---- Adjust these if your dataset folder name changes ----
INPUT_ROOT = Path("/kaggle/input")  # /kaggle/input/preprocessed-bonnfcd-flair -->caused error !!
RAW_SRC    = INPUT_ROOT / "nnUNet_raw_data_base/nnUNet_raw"
PP_SRC     = INPUT_ROOT / "nnUNet_preprocessed"
DATASET_ID = 2  # <--- ID of the dataset to use
DATASET_NAME = "BonnFCD"  # <--- Name of the dataset
DATASET_DIR = f"Dataset{DATASET_ID:03d}_{DATASET_NAME}_FLAIR"

# Where nnU-Net expects to READ/WRITE
os.environ["nnUNet_raw"]          = "/kaggle/working/nnUNet_raw_data"
os.environ["nnUNet_preprocessed"] = "/kaggle/working/nnUNet_preprocessed"
os.environ["nnUNet_results"]      = "/kaggle/working/nnUNet_results"

RAW_DST = Path(os.environ["nnUNet_raw"])
PP_DST  = Path(os.environ["nnUNet_preprocessed"])
RES_DST = Path(os.environ["nnUNet_results"])

FORCE_REPLAN = False

print("INPUT_ROOT:", INPUT_ROOT)
print("RAW_SRC   :", RAW_SRC / DATASET_DIR)
print("PP_SRC    :", PP_SRC / DATASET_DIR)
print("RAW_DST   :", RAW_DST)
print("PP_DST    :", PP_DST)
print("RES_DST   :", RES_DST)
print("DATASET_DIR:", DATASET_DIR)

INPUT_ROOT: /kaggle/input
RAW_SRC   : /kaggle/input/nnUNet_raw_data_base/nnUNet_raw/Dataset002_BonnFCD_FLAIR
PP_SRC    : /kaggle/input/nnUNet_preprocessed/Dataset002_BonnFCD_FLAIR
RAW_DST   : /kaggle/working/nnUNet_raw_data
PP_DST    : /kaggle/working/nnUNet_preprocessed
RES_DST   : /kaggle/working/nnUNet_results
DATASET_DIR: Dataset002_BonnFCD_FLAIR


## 2. Data Preparation
### 2.1 Copy RAW / PREPROCESSED data

In [ ]:
RAW_DST.mkdir(parents=True, exist_ok=True)
PP_DST.mkdir(parents=True, exist_ok=True)
RES_DST.mkdir(parents=True, exist_ok=True)

# Copy RAW
src = RAW_SRC / DATASET_DIR
dst = RAW_DST / DATASET_DIR
if not dst.exists():
    print("Copying RAW dataset to working directory...")
    shutil.copytree(src, dst)
else:
    print("RAW already present at:", dst)

# Copy PREPROCESSED if exists
pp_src_ds = PP_SRC / DATASET_DIR
pp_dst_ds = PP_DST / DATASET_DIR
if pp_src_ds.exists() and not FORCE_REPLAN:
    if not pp_dst_ds.exists():
        print("Copying PREPROCESSED to working directory...")
        shutil.copytree(pp_src_ds, pp_dst_ds)
    else:
        print("PREPROCESSED already present at:", pp_dst_ds)
else:
    if FORCE_REPLAN:
        print("FORCE_REPLAN=True → will run plan_and_preprocess.")
    else:
        print("No preprocessed input found; will run plan_and_preprocess.")

# 2) Preflight checks (dataset structure + geometry)

DS_ROOT = RAW_DST / DATASET_DIR
assert DS_ROOT.exists(), f"Dataset not found at {DS_ROOT}"

for folder in ["imagesTr", "labelsTr"]:
    assert (DS_ROOT / folder).exists(), f"Missing folder: {folder}"

dj_path = DS_ROOT / "dataset.json"
with open(dj_path) as f:
    dj = json.load(f)
print("channel_names:", dj.get("channel_names"))
print("labels:", dj.get("labels"))

# check file pairs
labels = sorted((DS_ROOT / "labelsTr").glob("*.nii*"))
for lbl in labels[:5]:
    stem = lbl.name.replace(".nii.gz", "").replace(".nii", "")
    img_path = (DS_ROOT / "imagesTr" / f"{stem}_0000.nii")
    if not img_path.exists():
        img_path = (DS_ROOT / "imagesTr" / f"{stem}_0000.nii.gz")
    img = nib.load(str(img_path))
    seg = nib.load(str(lbl))
    assert img.shape == seg.shape, f"Shape mismatch for {stem}"

print(f"✅ Preflight OK. Checked {len(labels[:5])} sample pairs.")

Copying RAW dataset to working directory...
Copying PREPROCESSED to working directory...
channel_names: {'0': 'FLAIR'}
labels: {'background': 0, 'lesion': 1}
✅ Preflight OK. Checked 5 sample pairs.


## 3. Configuration
### 3.1 Disable torch.compile

In [ ]:
os.environ["NNUNET_USE_TORCH_COMPILE"] = "0"   # <- main fix
os.environ["TORCH_COMPILE_DISABLE"] = "1"       # <- extra safety
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch._dynamo
torch._dynamo.config.suppress_errors = True

print("Torch Compile disabled. Ready to train nnU-Net safely on Tesla P100 ✅")

Torch Compile disabled. Ready to train nnU-Net safely on Tesla P100 ✅


## 4. Training Execution
### 4.1 Start Training (Fold 0)

In [ ]:
# Train single fold
FOLD=0
!nnUNetv2_train "$(printf '%03d' $DATASET_ID)" 3d_fullres $FOLD


############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2025-12-07 15:36:09.045076: Using torch.compile...
2025-12-07 15:36:10.979183: do_dummy_2d_data_aug: False
2025-12-07 15:36:10.979817: Creating new 5-fold cross-validation split...
2025-12-07 15:36:10.982501: Desired fold for training: 0
2025-12-07 15:36:10.982586: This split has 45 training and